# Prevención Fuga de Clientes
El enfoque de la prevención es disminuir el impacto latente de la posibilidad de que alguno de los clientes deje de comprar productos. 
Con este fin se realiza la estrategia en 3 sentidos: <br>
**1.Motor de Recomendación** <br>
**Objetivo:** Aumentar la penetración de los productos a los clientes con el fin de fortalecer la relación comercial y su fidelización. <br>
Este enfoque hace parte de la primera parte del notebook, con el objetivo de generar un motor de recomendación con el cual se facilite la venta de nuevos productos por medio de estrategia de datos. <br>
*Notebook 1: Motor de recomendación* <br>

**2.Nueva segmentación de información con base en los datos de los clientes y sus transacciones** <br>
**Objetivo:** Generar nueva información de los clientes, con el objetivo de realizar una mejor gestión de estos y plantear estrategias con base en esta nueva característica de información desarrollada. <br>
El segundo enfoque es a través de una nueva segmentación, que permita de igual forma identificar no solo como y que es posible comprar, sino generar nuevas dimensiones de análisis, para que los equipos de business intelligence, comerciales y financieros, tengan más herramientas durante sus estrategias de negocio. <br>
*Notebook 2: Segmentación de datos* <br>

**3.Predecir la Fuga de clientes** <br>
**Objetivo:** Desarrollar un marcaje de fuga de clientes con base en la historia transaccional de cliente, permitiendo esto entrenar un modelo de Machine Learning para predecir la posible baja de clientes. <br>
El Tercer enfoque tiene dos frentes de trabajo, el primero permite establecer a través de variables de tendencias de compra y montos, si el cliente se encuentra "vivo" o no. Con base en este resultado, se activa el segundo frente, el cual a través de algoritmos de Machine Learning permite predecir la fuga de clientes con base en el marcaje realizado en la primera etapa. <br>
*Notebook 3: Predicción Fuga Clientes* <br>


### Fuente de datos
Los datos usando durante la tesis son de dominio publico y hacen referencia a un comercio de Brasil. <br>
Estos datos han sido modelados en un modelo dimensional con el fin de mejorar el performance del análisis del actual tesis y permitir generar modelos de reporting/dashboard con herramientas de BI.<br>
Los datos han sido migrados a la plataforma github con el fin que sean de dominio publico. <br>


##1. Motor de Recomendación

Durante esta etapa se realizarán dos enfoques de recomendación, el primero con base en productos y el segundo con base en el comportamiento de clientes.


### 1.1 Carga de librerias y datos

In [6]:
!pip install turicreate
# librerias requeridas
import turicreate as tc
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
%matplotlib inline
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import time
from sklearn.model_selection import train_test_split


In [7]:
## Localización y carga de los datos
url = 'https://github.com/masdatascience/TFM-AI/blob/master/data_model_completo.xlsx?raw=true'
datos_modelo = pd.ExcelFile(url)
datos_facturacion = pd.read_excel(datos_modelo, sheet_name='fac_txn')
datos_producto = pd.read_excel(datos_modelo, sheet_name='product')
datos_clientes = pd.read_excel(datos_modelo, sheet_name='customer')

# Se elimian los datos nulos de las dimensiones importantes del analisis que son productos y clientes
datos_facturacion = datos_facturacion.dropna(subset=['customer_id'])
datos_facturacion = datos_facturacion.dropna(subset=['product_id'])
# Se elimina posibles chargeback que se presenten
datos_facturacion = datos_facturacion.loc[datos_facturacion['total_value'] > 0]

print("Facturación:"+ str(datos_facturacion.shape) + " Clientes:"+str(datos_clientes.shape)+" Productos:"+str(datos_producto.shape))

Facturación:(112650, 21) Clientes:(96352, 4) Productos:(32951, 10)


In [0]:
from sklearn.preprocessing import LabelEncoder
enconder = LabelEncoder()
datos_facturacion['ID_CLIENTE'] = enconder.fit_transform(datos_facturacion['customer_id'])
datos_facturacion['ID_PRODUCTO'] = enconder.fit_transform(datos_facturacion['product_id'])


In [0]:
grupo1 = ['ID_CLIENTE', 'ID_PRODUCTO', 'total_value']

# Enfoque 
datos = datos_facturacion.loc[:, grupo1] 

### 1.2 Preparación de los datos 
En esta sección se estarán preparando tres enfoques de matrices con el fin de permitir desarrollar los motores de recomendación. La primera es la relación entre el cliente y los productos, la siguiente es una matriz normalizada y la última integra valores dummies para la comparación . 

#### 1.2.1 Relación Cliente - Producto



In [10]:
# Obtengo la Matriz Relación Cliente - Producto , y la normalizo
matri = datos.tail(27000)
matrix_usuario_producto =pd.crosstab(matri['ID_CLIENTE'],matri['ID_PRODUCTO'],values=matri['total_value'], aggfunc='sum')
matrix_usuario_producto_norm = (matrix_usuario_producto-matrix_usuario_producto.min())/(matrix_usuario_producto.max()-matrix_usuario_producto.min())
d = matrix_usuario_producto_norm.reset_index() 
d.index.names = ['total_frecuencia'] 
data_norm = pd.melt(d, id_vars=['ID_CLIENTE'], value_name='total_frecuencia').dropna()
data_norm['ID_CLIENTE'] = data_norm['ID_CLIENTE'].astype(str)
data_norm['ID_PRODUCTO'] = data_norm['ID_PRODUCTO'].astype(str)
print("Matriz normalizada:" +str(data_norm.shape))

Matriz normalizada:(15019, 3)


#### 1.2.2 Relación Cliente Producto - Contraste

In [11]:
datos_dummy = datos_facturacion.copy()
datos_dummy['total_dummy'] = 1
datos_dummy['ID_CLIENTE'] = datos_dummy['ID_CLIENTE'].astype(str)
datos_dummy['ID_PRODUCTO'] = datos_dummy['ID_PRODUCTO'].astype(str)
print("Matriz Dummy:"+ str(datos_dummy.shape))

Matriz Dummy:(112650, 24)


#### 1.2.3 Se generan datos de entrenamiento y pruebas

In [0]:
train_data, test_data  = train_test_split(data_norm, test_size=0.3 ,random_state=1)
train_data_dummy, test_data_dummy = train_test_split(datos_dummy, test_size=0.3 ,random_state=1)

#### 1.2.4  Se define función para ejecutar los tipos de motores de recomendación

In [0]:
# Se define una función que retorne un modelo resultado segun el enfoque que se requiere:
# 1. Popularity : Es el item que más se a consumido en todos los clientes
# 2. cosine :
# 3. Pearson: 
def modelo_requerido(modelo,datos ,campo ):

    train_data=tc.SFrame(datos)
    print("Modelo solicitado:"+ str(modelo))
    if modelo == 'producto':
        return  tc.popularity_recommender.create(train_data,user_id='ID_CLIENTE', item_id='ID_PRODUCTO', target=campo)
    elif modelo == 'distancia_coseno':
        return tc.item_similarity_recommender.create(train_data,user_id='ID_CLIENTE', item_id='ID_PRODUCTO', target=campo,similarity_type='cosine')
    elif modelo == 'pearson':
        return tc.item_similarity_recommender.create(train_data,user_id='ID_CLIENTE', item_id='ID_PRODUCTO', target=campo,similarity_type='pearson')
    else : 
        print("ERROR: Modelo solicitado no existe")

#Obtiene las recomendaciones de forma visual para los clientes
def productos_recomendado(modelo, usuarios, cantidad_recomendaciones):
     modelo.recommend(users=list(usuarios), k=cantidad_recomendaciones).print_rows(cantidad_recomendaciones)

#Evalua los modelos generados 
def evaluacion_modelos(modelo, test):
  
  if modelo == 'normalizado':
        modelos_generados= [fac_producto, fac_coseno, fac_pearson]
        nombres_modelos = ['Modelo basado en contenido', 'Modelo Colaborativo - Distancia Coseno', 'Modelo Colaborativo - Pearson']
        test_data= tc.SFrame(test) 
        eval_counts = tc.recommender.util.compare_models(test_data, modelos_generados, model_names=nombres_modelos)
  elif modelo == 'dummy':
        modelos_generados= [dum_producto, dum_coseno, dum_pearson]
        nombres_modelos = ['Modelo Dummy basado en contenido', 'Modelo Dummy Colaborativo - Distancia Coseno', 'Modelo Dummy Colaborativo - Pearson']
        test_data= tc.SFrame(test) 
        eval_counts = tc.recommender.util.compare_models(test_data, modelos_generados, model_names=nombres_modelos)
  else : 
        print("ERROR: Modelo solicitado no existe")
      
#Exporta cinco  recomendaciones por cliente en google Drive      
def exportarRecomendaciones(modelo, clientes):

  lista_clientes =  list(clientes)
  recomendaciones = modelo.recommend(users=lista_clientes, k=5)
  resultado = recomendaciones.to_dataframe()
  resultado['id_producto_recomendado'] = resultado.groupby(['customer_id'])['product_id'].transform(lambda x: ';'.join(x.astype(str)))
  exportar = resultado[['customer_id', 'id_producto_recomendado']].drop_duplicates().sort_values('customer_id').set_index('customer_id')
  exportar.to_csv('recomendaciones.csv')
  !cp recomendaciones.csv drive/My\ Drive/TFM-AI/
  print("Se han exportado las recomendaciones en Google drive")


### 1.3 Modelo de recomendación basado en Contenido
Este modelo genera recomendaciones con base en la relación de los clientes y sus productos

In [14]:
# Se generan las recomendaciones por dataset
fac_producto = modelo_requerido('producto',train_data,'total_frecuencia' )
dum_producto = modelo_requerido('producto' ,train_data_dummy, 'total_dummy')

Modelo solicitado:producto


Preparing data set.

Data has 10513 observations with 10299 users and 3325 items.

Data prepared in: 0.036659s

10513 observations to process; with 3325 unique items.

...

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 78855 observations with 69445 users and 26898 items.

Data prepared in: 0.150769s

78855 observations to process; with 26898 unique items.

In [15]:
# se obtienen identificadores de clientes 
data_norm.ID_CLIENTE.unique()

array(['13297', '89261', '40208', ..., '58764', '35851', '67581'],
      dtype=object)

In [16]:
# Se generan las recomendaciones para un Cliente
print("=============================================")  
print("Metodo: Basado en contenido - DS Normalizado")
print("============================================")  
productos_recomendado(fac_producto,'48821',5)
productos_recomendado(fac_producto,'66843',5)

print("============================================")  
print("Metodo: Basado en contenido - DS Referencia")
print("============================================")  
productos_recomendado(dum_producto,'48821',5)
productos_recomendado(dum_producto,'66843',5)

Metodo: Basado en contenido - DS Normalizado
+------------+-------------+-------+------+
| ID_CLIENTE | ID_PRODUCTO | score | rank |
+------------+-------------+-------+------+
|     4      |    32148    |  1.0  |  1   |
|     4      |    32385    |  1.0  |  2   |
|     4      |     531     |  1.0  |  3   |
|     4      |     8019    |  1.0  |  4   |
|     4      |    24153    |  1.0  |  5   |
+------------+-------------+-------+------+
[25 rows x 4 columns]

+------------+-------------+-------+------+
| ID_CLIENTE | ID_PRODUCTO | score | rank |
+------------+-------------+-------+------+
|     6      |    32148    |  1.0  |  1   |
|     6      |    32385    |  1.0  |  2   |
|     6      |     531     |  1.0  |  3   |
|     6      |     8019    |  1.0  |  4   |
|     6      |    24153    |  1.0  |  5   |
+------------+-------------+-------+------+
[25 rows x 4 columns]

Metodo: Basado en contenido - DS Referencia
+------------+-------------+-------+------+
| ID_CLIENTE | ID_PRODUCTO | 

### 1.4 Motor de recomendación Colaborativo
Este modelo genera recomendaciones con base en el comportamiento de compra que tienen los usuarios. <br>
Con este fin se realizan pruebas con dos enfoques, la distancia coseno y pearson para evaluar cual es el que da mejores resultados

In [17]:
# Se generan las recomendaciones por dataset
fac_coseno = modelo_requerido('distancia_coseno',train_data,'total_frecuencia' )
dum_coseno = modelo_requerido('distancia_coseno' ,train_data_dummy, 'total_dummy')

Modelo solicitado:distancia_coseno


Preparing data set.

Data has 10513 observations with 10299 users and 3325 items.

Data prepared in: 0.039998s

Training model from provided data.

...

| 4.58s                               | 100              | 26898           |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 4.60503s

In [18]:
print("=============================================")  
print("              DISTANCIA COSENO ")
print("============================================")  
# Se generan las recomendaciones para un Cliente
print("=============================================")  
print("Metodo: Colaborativo - DS Normalizado")
print("============================================")  
fac_coseno.recommend(users=list(['48821','40208']), k=5).print_rows(10)

print("============================================")  
print("Metodo: Colaborativo - DS Referencia")
print("============================================")  
dum_coseno.recommend(users=list(['48821','40208']), k=5).print_rows(10)

              DISTANCIA COSENO 
Metodo: Colaborativo - DS Normalizado
+------------+-------------+------------------------+------+
| ID_CLIENTE | ID_PRODUCTO |         score          | rank |
+------------+-------------+------------------------+------+
|   48821    |    29513    |  0.00263270378112793   |  1   |
|   48821    |     7131    | 0.0018765747547149659  |  2   |
|   48821    |    31399    | 0.0016767871379852295  |  3   |
|   48821    |    13831    | 0.0007235562801361084  |  4   |
|   48821    |     8227    | 0.00044424653053283693 |  5   |
|   40208    |    29513    |  0.00263270378112793   |  1   |
|   40208    |     7131    | 0.0018765747547149659  |  2   |
|   40208    |    31399    | 0.0016767871379852295  |  3   |
|   40208    |    13831    | 0.0007235562801361084  |  4   |
|   40208    |     8227    | 0.00044424653053283693 |  5   |
+------------+-------------+------------------------+------+
[10 rows x 4 columns]

Metodo: Colaborativo - DS Referencia
+------------+--

In [19]:
# Se generan las recomendaciones por dataset
fac_pearson= modelo_requerido('pearson',train_data,'total_frecuencia' )
dum_pearson = modelo_requerido('pearson' ,train_data_dummy, 'total_dummy')

Modelo solicitado:pearson


Preparing data set.

Data has 10513 observations with 10299 users and 3325 items.

Data prepared in: 0.037368s

Training model from provided data.

...

| 4.32s                               | 100              | 26898           |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 4.35216s

In [20]:
print("=============================================")  
print("          CORRELACION PEARSON ")
print("============================================")  
# Se generan las recomendaciones para un Cliente
print("=============================================")  
print("Metodo: Colaborativo - DS Normalizado")
print("=============================================") 
fac_pearson.recommend(users=list(['48821','66843']), k=5).print_rows(10)

print("============================================")  
print("Metodo: Colaborativo - DS Referencia")
print("============================================") 
dum_pearson.recommend(users=list(['48821','66843']), k=5).print_rows(10) 

          CORRELACION PEARSON 
Metodo: Colaborativo - DS Normalizado
+------------+-------------+-------+------+
| ID_CLIENTE | ID_PRODUCTO | score | rank |
+------------+-------------+-------+------+
|   48821    |    32148    |  1.0  |  1   |
|   48821    |    32385    |  1.0  |  2   |
|   48821    |     531     |  1.0  |  3   |
|   48821    |     8019    |  1.0  |  4   |
|   48821    |    24153    |  1.0  |  5   |
|   66843    |    32148    |  1.0  |  1   |
|   66843    |    32385    |  1.0  |  2   |
|   66843    |     531     |  1.0  |  3   |
|   66843    |     8019    |  1.0  |  4   |
|   66843    |    24153    |  1.0  |  5   |
+------------+-------------+-------+------+
[10 rows x 4 columns]

Metodo: Colaborativo - DS Referencia
+------------+-------------+-------+------+
| ID_CLIENTE | ID_PRODUCTO | score | rank |
+------------+-------------+-------+------+
|   48821    |     3222    |  0.0  |  1   |
|   48821    |    14119    |  0.0  |  2   |
|   48821    |    20754    |  0.0  

### 1.5 Evaluación de Modelos 
Con el fin de establecer cual es el mejor modelo entre los tres probados, se realiza una validación de estos de forma conjunta 

In [21]:
evaluacion_modelos('normalizado',test_data)

PROGRESS: Evaluate model Modelo basado en contenido


recommendations finished on 1000/4467 queries. users per second: 43639.5

recommendations finished on 2000/4467 queries. users per second: 42957.2

recommendations finished on 3000/4467 queries. users per second: 43013.8

recommendations finished on 4000/4467 queries. users per second: 43231.1

...

recommendations finished on 1000/4467 queries. users per second: 36893.6

recommendations finished on 2000/4467 queries. users per second: 35496.9

recommendations finished on 3000/4467 queries. users per second: 36140.2

recommendations finished on 4000/4467 queries. users per second: 36911.9


Precision and recall summary statistics by cutoff
+--------+------------------------+------------------------+
| cutoff |     mean_precision     |      mean_recall       |
+--------+------------------------+------------------------+
|   1    | 0.00022386389075442135 | 0.00022386389075442135 |
|   2    | 0.00022386389075442116 | 0.0004477277815088423  |
|   3    | 0.00022386389075442132 |  0.000671591672263264  |
|   4    | 0.00022386389075442127 | 0.0008954555630176851  |
|   5    | 0.00022386389075442127 | 0.0011193194537721073  |
|   6    | 0.00022386389075442127 |  0.001343183344526527  |
|   7    | 0.0002238638907544212  | 0.0015670472352809486  |
|   8    | 0.00022386389075442135 | 0.0017909111260353708  |
|   9    | 0.00019899012511504126 | 0.0017909111260353704  |
|   10   | 0.00020147750167897924 | 0.0020147750167897917  |
+--------+------------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 0.6023458116591319

Per User RMSE (best)
+------------+--

In [22]:
evaluacion_modelos('dummy',test_data_dummy)  

PROGRESS: Evaluate model Modelo Dummy basado en contenido


recommendations finished on 1000/31733 queries. users per second: 10335.4

recommendations finished on 2000/31733 queries. users per second: 10214.6

recommendations finished on 3000/31733 queries. users per second: 10154.2

recommendations finished on 4000/31733 queries. users per second: 10229.9

...

recommendations finished on 28000/31733 queries. users per second: 6446.11

recommendations finished on 29000/31733 queries. users per second: 6450.45

recommendations finished on 30000/31733 queries. users per second: 6448.37

recommendations finished on 31000/31733 queries. users per second: 6451.65


Precision and recall summary statistics by cutoff
+--------+------------------------+-----------------------+
| cutoff |     mean_precision     |      mean_recall      |
+--------+------------------------+-----------------------+
|   1    |          0.0           |          0.0          |
|   2    |          0.0           |          0.0          |
|   3    | 2.1008624040168487e-05 | 6.302587212050537e-05 |
|   4    | 1.575646803012636e-05  | 6.302587212050544e-05 |
|   5    | 1.260517442410108e-05  | 6.302587212050541e-05 |
|   6    | 1.0504312020084247e-05 | 6.302587212050522e-05 |
|   7    | 9.003696017215075e-06  | 6.302587212050503e-05 |
|   8    |  7.87823401506318e-06  | 6.302587212050544e-05 |
|   9    | 1.0504312020084311e-05 |  9.4538808180758e-05  |
|   10   | 9.453880818075787e-06  | 9.453880818075812e-05 |
+--------+------------------------+-----------------------+
[10 rows x 3 columns]


Overall RMSE: 1.0

Per User RMSE (best)
+------------+------+-------+
| ID_CLIENTE | 

### 1.6 Exportar recomendaciones
Con los resultado obtenidos podemos identificar que el mejor modelo es el Pearson, por tal motivo se exporta su información . 

In [30]:
from google.colab import files
lista_clientes =  list(data_norm.ID_CLIENTE.unique())
recomendaciones = fac_coseno.recommend(users=lista_clientes, k=5) 
resultado = recomendaciones.to_dataframe()

recommendations finished on 1000/14598 queries. users per second: 47463.1

recommendations finished on 2000/14598 queries. users per second: 49252.6

recommendations finished on 3000/14598 queries. users per second: 49868.7

recommendations finished on 4000/14598 queries. users per second: 50110.2

recommendations finished on 5000/14598 queries. users per second: 50504.5

...

recommendations finished on 10000/14598 queries. users per second: 50563

recommendations finished on 11000/14598 queries. users per second: 50665.1

recommendations finished on 12000/14598 queries. users per second: 50589.8

recommendations finished on 13000/14598 queries. users per second: 50011.3

recommendations finished on 14000/14598 queries. users per second: 48614.8

In [31]:
resultado.to_csv('recomendaciones_prod_cliente.csv')
# verifico el archivo 
!dir

recomendaciones_prod_cliente.csv  sample_data


In [0]:
files.download('recomendaciones_prod_cliente.csv')